In [1]:
%load_ext autoreload
%autoreload 2

import os 
import shutil
import numpy as n
from datetime import date
from matplotlib import pyplot as plt
import time
import napari

In [2]:
os.chdir('/home/ali/packages/s2p-lbm/')

from suite3d.job import Job
from suite3d import lbmio, utils, ui
from suite3d import tiff_utils as tfu
from suite3d import file_utils as flu

In [3]:
job_params = {
    'subjects_dir' :   '/mnt/a2',
    'subject' :        'AH007',
    'date' :           '2022-03-28',
    'expnum' :         [2,4,5],
    'planes' :         n.arange(0,30),
    
    'notch_filt' :     {'f0' : 200, 'Q' : 5},
    
    'n_init_files' :   3,
    'init_file_pool' : None,
    'init_file_sample_method': 'random',
    
    'subtract_crosstalk' : True,
    'override_crosstalk' : None,
    'crosstalk_n_planes' : 10, 
    
    'nonrigid' : True,
    'smooth_sigma' : 1.15,
    'maxregshift' : 0.15,
    'reg_filter_pcorr' : 1,
    'generate_sample_registered_bins' : False,

    'dtype' : n.float32,
    'tif_batch_size' : 1,
}



In [4]:
tifs, si_params, exp_str = flu.find_exp(job_params['subjects_dir'], job_params['subject'],
                                        job_params['date'], job_params['expnum'])

SI ROI pix count in x is 182, which is impossible, setting it to 145


In [5]:
job_params['fs'] = si_params['vol_rate']
job_params['notch_filt']['line_freq'] = si_params['line_freq']

In [6]:
# job.save_frame_counts()

In [7]:
# tifs_nofilt = lbmio.load_and_stitch_tifs(job.tifs[:2], planes = n.arange(10),filt=None)

# tifs_filt = lbmio.load_and_stitch_tifs(job.tifs[:2], planes=n.arange(10), filt = job_params['notch_filt'])

# v = napari.view_image(tifs_nofilt, name='no filt')
# v.add_image(tifs_filt, name='filt')

In [8]:
# Create the job
job = Job('/mnt/md0/runs',exp_str + '-filtered', tifs = tifs,
          params=job_params, create=False, overwrite=False, verbosity = 3)

   Found and loaded params from /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5-filtered/params.npy


In [9]:
# optional parameters for initialization
# load 1 file to initialize, and select 200 frames randomly to do initalization on
job.params['n_init_files'] = 3
job.params['init_n_frames'] = None


In [10]:
# %%time
# summary = job.run_init_pass()

In [11]:
# split the large tiffs into files of size 100 after registration
job.params['split_tif_size'] = 100

In [12]:
# %%time
# # job.register_gpu()

In [13]:
mov_full = job.get_registered_movie('registered_fused_data', 'fused')
# im_full = mov_full[:,:200].mean(axis=1).compute()

In [14]:
# find crop values that minimize dark zones - check planes 0 and 15 in the following cell to 
# make sure you're not cutting out parts of the brain
crop = ((0,18), (100,1100), (50, 900))

In [15]:
# number of SVD components to use when calculating the correlation map
# lower number means more denoising, but if it's too low you will start losing cells!
# This can't be larger than the number of svd components you used in the decomposition above
job.params['n_svd_comp'] = 50

# spatial filter sizes for neuropil subtraction, and cell detection
# npil_filt is a low_pass filter that attempts to remove any features larger than the filter size (neuropil!)
# conv_filt_xy is a high_pass filter that amplifies any features that are smaller than ~2x the filter size (cells!)
# these values worked well for me with ~4um xy pixel spacing and ~15 um z pixel spacing, for detecting mouse somata
# When you change resolution, or if you're trying to detect smaller things, you will need to adjust these values
# because the units here are _pixels_, not microns!
job.params['conv_filt_type'] = 'gaussian'
job.params['conv_filt_xy'] = 1.0
job.params['conv_filt_z'] = 1.0
job.params['npil_filt_type'] = 'unif'
job.params['npil_filt_xy'] = 8.0
job.params['npil_filt_z']=  1.5

# normalization exponent, should be around 1. 
# If you find blood vessels or the background being too bright in the correlation map, reduce it to ~0.7-0.8! 
job.params['sdnorm_exp']= 0.8

# threshold applied to the normalized, filtered movie before it is accumulated into the correlation map
# if you increase it, the background will become darker (which is good!), however at some point you will
# start excluding dimmer cells (which is bad!)
job.params['intensity_thresh'] : 0.4

## Compute parameters 
# number of frames to compute at one iteration 
# (any value above ~100-200 shouldn't affect results, 
# decrease if you have RAM issues or if SVD reconstruction gets stuck on "Sending all blocks to dask to compute")
job.params['t_batch_size'] = 300
# number of processors to use when calculating the correlation map
job.params['n_proc_corr'] = 70
# number of frames per smaller batch within the batch, should be ~t_batch_size / n_proc_corr, but above ~5
job.params['mproc_batchsize'] = 5


In [16]:
mov_crop = mov_full[crop[0][0]:crop[0][1], :, crop[1][0]:crop[1][1], crop[2][0]:crop[2][1]]

In [18]:
# save clips from retinotopy for analysis later
retinotopy_dir = job.make_new_dir('retinotopy_data')
t0, t1 = job.get_exp_frame_idxs(4)
# for plane_idx in (0,5,10):
#     mov_save = mov_crop[plane_idx,t0:t1].compute()
#     n.save(os.path.join(retinotopy_dir, 'plane_%02d.npy' % plane_idx), mov_save)
#     print("Saved")

      Found dir /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5-filtered/retinotopy_data with tag retinotopy_data


In [21]:
mov_mean = mov_crop[:13][:,t0:t1].mean(axis=0)

In [22]:
xx = mov_mean.compute()

In [23]:
n.save(os.path.join(retinotopy_dir, 'plane_mean.npy'), xx)

dask.array<mean_agg-aggregate, shape=(10194, 1000, 850), dtype=float32, chunksize=(100, 1000, 850), chunktype=numpy.ndarray>

In [41]:
mov_crop.shape

(18, 10194, 1000, 850)

In [15]:
%%time
# uncomment below to load svd_info for and svd you did earlier if you are re-running this notebook
# svd_info = n.load(os.path.join(job.dirs['svd_cropped'], 'svd_info.npy'), allow_pickle=True).item()
# (vmap, mean_img, max_img), __, __ = job.calculate_corr_map(mov = mov_crop)

CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 3.1 µs


In [17]:
# # Pick the parameters you want to sweep, and enter them in the tuples.
# # It will do all combinations of parameters, so if you have many parameters it will be many combinations! 
# job.params['t_batch_size'] = 300
# params_to_sweep = {
#     'intensity_thresh' : (0.25, 0.4),
#     'n_svd_comp' : (50,), # if you have multiple values here, make sure you pass mov=svd_info
#     'conv_filt_xy': (0.7,1.0,),
#     'npil_filt_xy': (3.0,5.0,8.0),
#     'conv_filt_z' : (1.0,1.5),
#     'npil_filt_z' : (1.0,1.5,),
#     'sdnorm_exp' : (0.8, 1.0)
# }

# sweep_summary = job.sweep_params(params_to_sweep, mov = mov_crop,testing_dir_tag='sweep-nosvd',
#                            do_vmap=True, all_combinations=True, n_test_iters = 1)
# # use this to access older sweep results
# # sweep_summary = n.load(os.path.join(job.dirs['sweep-full'], 'sweep_summary.npy'),allow_pickle=True).item()
# job.vis_vmap_sweep(sweep_summary)

In [17]:
vmap = job.load_iter_results(-1)['vmap']

   Loading from /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5-filtered/iters/batch0033


In [19]:
%%time
job.params['detection_timebin'] = 1 
job.params['detection_time_crop'] = (None,None)
job.params['max_pix'] = 250
job.params['peak_thresh'] = 27.0

job.params['activity_thresh'] = 10.0
job.params['percentile'] = 99.5
job.params['extend_thresh'] = 0.15
job.params['max_iter'] = 10000 # maximum number of ROIs detected in given patch
# combined_dir = job.patch_and_detect(combined_name='combined-full')

CPU times: user 2 µs, sys: 4 µs, total: 6 µs
Wall time: 9.06 µs


In [20]:
# job.compute_npil_masks(combined_dir)

In [21]:
# traces = job.extract_and_deconvolve(stats_dir = combined_dir, mov=mov_crop, crop=False)

In [23]:
combined_dir = job.dirs['detection-combined-full']

In [26]:
mov_crop.shape

(18, 10194, 1000, 850)

In [30]:
mov_sample = mov_crop[:,:200].compute()

In [31]:
napari.view_image(mov_sample)

Viewer(camera=Camera(center=(0.0, 499.5, 424.5), zoom=1.0495015384615385, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(8.0, 99.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=4, ndisplay=2, last_used=0, range=((0.0, 18.0, 1.0), (0.0, 200.0, 1.0), (0.0, 1000.0, 1.0), (0.0, 850.0, 1.0)), current_step=(8, 99, 499, 424), order=(0, 1, 2, 3), axis_labels=('0', '1', '2', '3')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'mov_sample' at 0x7f6a7d3189a0>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x7f6df17e75e0>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap={})

In [25]:
outputs = ui.load_outputs(combined_dir, load_traces=True)
v,layers = ui.create_ui(outputs)
ui.add_callbacks_to_ui(v,layers,outputs,savedir=combined_dir)

Found old curated iscell with 60744 of 60745 marked as cells
Saving old iscell_curated to backup path /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5-filtered/detection/combined-full/iscell_curated_old_25-10-23_11-36-01.npy
Cells
Found old curated + slider-ed iscell with 60744 of 60745 marked as cells
Saving old iscell_curated to backup path /mnt/md0/runs/s3d-AH007_2022-03-28_2-4-5-filtered/detection/combined-full/iscell_curated_slider_old_25-10-23_11-36-02.npy
('vmap_peak', (27.000048, 31.11153259277344), 'peak_val', <function add_curation_sliders.<locals>.<lambda> at 0x7f6a93b6e820>)
50421, 50421 cells valid
Cells
32734, 32734 cells valid
Cells
Updating cell 6074
32733, 32733 cells valid
Cells
Updating cell 6131
32732, 32732 cells valid
Cells
Updating cell 6398
32731, 32731 cells valid
Cells
Updating cell 10139
32730, 32730 cells valid
Cells
Updating cell 11105
32729, 32729 cells valid
Cells
Updating cell 10642
32728, 32728 cells valid
Cells
Updating cell 17194
32727, 32727 cells valid
Cell